### Authenticate
Get the client id and secret from env vars for prod.
In dev we can create a new client each time

In [ ]:
import { registerSystem } from '../v1-to-v2-data-migration/helpers/gqlHandlers.ts'
import { authenticate, getTokenForSystemClient } from '../v1-to-v2-data-migration/helpers/authentication.ts'
import { CLIENT_ID, CLIENT_SECRET } from '../v1-to-v2-data-migration/helpers/vars.ts'
import { ADMIN_USERNAME, ADMIN_PASSWORD } from '../v1-to-v2-data-migration/helpers/vars.ts'


let clientId: string | null = null
let clientSecret: string | null = null

if (CLIENT_ID && CLIENT_SECRET) {
  clientId = CLIENT_ID
  clientSecret = CLIENT_SECRET
} else {
  // For dev mode
  const adminToken = await authenticate(ADMIN_USERNAME, ADMIN_PASSWORD)
  const systemRegistration = await registerSystem(adminToken)

  clientId = systemRegistration.data.registerSystem.system.clientId
  clientSecret = systemRegistration.data.registerSystem.clientSecret
}

const sysToken = await getTokenForSystemClient(clientId, clientSecret)
sysToken

### Fetch data from CSVs

In [ ]:
import { csvToJson } from './helpers/csvHelpers.ts'

const pathToBirthCsv = './sourceData/Birth_Register.csv'
const pathToDeathCsv = './sourceData/Death_Register.csv'
const pathToMarriageCsv = './sourceData/Marriage_Register.csv'
const pathToAdoptionCsv = './sourceData/Adoption_Register.csv'
const pathToDeedpollCsv = './sourceData/Deedpoll.csv'

const csvData = {
  birth: await csvToJson(pathToBirthCsv),
  death: await csvToJson(pathToDeathCsv),
  marriage: await csvToJson(pathToMarriageCsv),
  adoption: await csvToJson(pathToAdoptionCsv),
  deedpoll: await csvToJson(pathToDeedpollCsv),
}


In [ ]:
import { GATEWAY } from '../v1-to-v2-data-migration/helpers/routes.ts'
import { locationsMap } from './lookupMappings/locations.ts'

export const getLocations = async (
  token: string,
  type: 'ADMIN_STRUCTURE' | 'HEALTH_FACILITY',
) => {
  const response = await fetch(
    `${GATEWAY}/location?type=${type}&_count=0&status=active`,
    {
      method: 'GET',
      headers: {
        'Content-Type': 'application/json',
        Authorization: `Bearer ${token}`,
      },
    },
  )
  if (!response.ok) {
    throw new Error(`Sync Locations failed: ${response.statusText}`)
  }
  return response
}

const locationsRes = await getLocations(sysToken, 'ADMIN_STRUCTURE')
const healthFacilitiesRes = await getLocations(sysToken, 'HEALTH_FACILITY')
const fhirLocations = await locationsRes.json()
const fhirHealthFacilities = await healthFacilitiesRes.json()
const locationCodes = fhirLocations.entry
  .concat(fhirHealthFacilities.entry)
  .map((loc: any) => ({
    id: loc.resource.id,
    name: loc.resource.name,
    code:
      loc.resource.identifier[0]?.value
        .replace('ADMIN_STRUCTURE_', '')
        .replace('HEALTH_FACILITY_', '') || null,
  }))


const updatedLocationsMap = locationsMap.map((loc: any) => ({
  ...loc,
  id: locationCodes.find((l: any) => l.code === loc.map)?.id || null,
}))
updatedLocationsMap

### Get all potential resolvers
Use only resolvers for used event fields to avoid nulls

In [ ]:
import { birthResolver, birthMetaData } from './mappings/birthResolver.ts'
import { transform } from './helpers/transform.ts'
import { bulkImport } from '../v1-to-v2-data-migration/helpers/gqlHandlers.ts'
import {
  batch,
  getIndexErrors,
} from '../v1-to-v2-data-migration/helpers/utils.ts'
import { generateRegistrationNumber } from './helpers/generators.ts'
import { CrvsEvent } from './helpers/types.ts'
import { BirthResolver } from './helpers/birthTypes.ts'

function nonNullObjectKeys(obj: Record<string, any>) {
  return Object.fromEntries(
    Object.entries(obj).filter(
      ([_, value]) => value !== null && value !== undefined && value !== '',
    ),
  )
}

const events: Array<CrvsEvent> = []

csvData.birth.forEach((birth) => {
  const declaration: Partial<Record<keyof typeof birthResolver, any>> = {}
  Object.entries(birthResolver).forEach(([eventField, dataField]) => {
    if (typeof dataField === 'function') {
      const data = dataField(birth, csvData, updatedLocationsMap)
      declaration[eventField as keyof typeof birthResolver] = data
    }
  })

  const meta: Partial<Record<keyof typeof birthMetaData, any>> = {}
  Object.entries(birthMetaData).forEach(([metaField, dataField]) => {
    const data = dataField(birth, csvData, updatedLocationsMap)
    meta[metaField as keyof typeof birthMetaData] = data
  })

  const user = 'f686c526-c6b5-41ed-b3cc-43b104fa5c03'
  //const location = '08260e22-bb67-4702-8760-86ec125e1079'
  const role = 'REGISTRAR' // Probably ok to hardcode this

  const locationId = locationCodes.find(
    (loc: any) => loc.code === meta.locationCode,
  )?.id

  const event = transform(
    nonNullObjectKeys(declaration),
    'birth',
    meta.registrationDate || new Date().toISOString(),
    user, //TODO change to lookup meta.registrar,
    role,
    locationId,
    birth.BIRTH_REF,
    generateRegistrationNumber(
      meta.locationCode || '',
      meta.registrationDate || new Date().toISOString(),
      'birth',
    ),
  )
  events.push(event)
})

const batches = batch(events, 1000)

for (const batch of [batches[0]]) {
  const res = await bulkImport(batch, sysToken)
  console.log(res)
  const errors = getIndexErrors(res)
  if (errors) {
    console.error('Errors during bulk import', errors)
  }
}


In [ ]:
import { reindex } from "../v1-to-v2-data-migration/helpers/gqlHandlers.ts";

const reindexResponse = await reindex(sysToken);
reindexResponse


### Output results


In [ ]:
console.log('🍞 Declarations succesfully migrated')
